In [1]:
import transformers
from transformers import *
import numpy as np
import torch

/Users/jstav/miniconda3/envs/nlu/lib/python3.9/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [2]:
name = 'facebook/bart-base'
generator = BartForConditionalGeneration.from_pretrained(name)
tokenizer = BartTokenizer.from_pretrained(name)


loading configuration file config.json from cache at /Users/jstav/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/config.json
Model config BartConfig {
  "_name_or_path": "bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "

In [96]:
prompt_1 = "Jacob is my friend. His name is <mask>."
prompt_2 = "Kevin is my friend. His name is <mask>."

In [63]:
# defines prompts
prompt_1 = """
I am happy. I am happy.
I am sad. I am sad.
I am sad. I am sad.
I am happy. I am happy.
I am sad. I am sad.
I am happy. I am happy.
I am happy. I am happy.
I am happy. I am happy.
I am happy. I am happy.
I am sad. I am sad.
I am happy. I am <mask>."""
prompt_2 = """
I am happy. I am happy.
I am sad. I am sad.
I am sad. I am sad.
I am happy. I am happy.
I am sad. I am sad.
I am happy. I am happy.
I am happy. I am happy.
I am happy. I am happy.
I am happy. I am happy.
I am sad. I am sad.
I am sad. I am <mask>."""

In [97]:
# tokenizes prompts
input_ids_1 = tokenizer.encode(prompt_1, return_tensors='pt')
input_ids_2 = tokenizer.encode(prompt_2, return_tensors='pt')

In [78]:
print(input_ids_1, len(input_ids_1[0]))
print(input_ids_2, len(input_ids_2[0]))

tensor([[    0, 41884,    16,   127,  1441,     4,   832,   766,    16, 50264,
             4,     2]]) 12
tensor([[    0, 21910,    16,   127,  1441,     4,   832,   766,    16, 50264,
             4,     2]]) 12


In [98]:
# encodes prompts
embed_matrix = generator.get_input_embeddings()
embeddings_1 = embed_matrix(input_ids_1)
embeddings_2 = embed_matrix(input_ids_2)

In [108]:
# averages embeddings
embeddings_avg = (embeddings_1 + 2 * embeddings_2) / 3

In [109]:
# chooses embeddings to use
inputs_embeds = embeddings_avg

In [110]:
# magic black-box code...
attention_mask = torch.ones(inputs_embeds.shape[:2], dtype=torch.long)
decoder_input_ids = torch.ones((inputs_embeds.shape[0], 1), dtype=torch.long)*generator.config.decoder_start_token_id
output_ids = generator.generate(attention_mask=attention_mask, decoder_input_ids=decoder_input_ids, inputs_embeds=inputs_embeds)

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

/Users/jstav/miniconda3/envs/nlu/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [111]:
# decodes output_ids
tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

'Kevin is my friend. His name is Kevin.'

In [112]:
# constants
NUM_BEAMS = 20
NUM_COMPLETIONS = 10

In [113]:
# black box
attention_mask = torch.ones(inputs_embeds.shape[:2], dtype=torch.long)
decoder_input_ids = torch.ones((inputs_embeds.shape[0], 1), dtype=torch.long)*generator.config.decoder_start_token_id

# samples with replacement (i.e. duplicates)
output_ids = generator.generate(attention_mask=attention_mask, decoder_input_ids=decoder_input_ids, inputs_embeds=inputs_embeds, num_beams=NUM_BEAMS, num_return_sequences=NUM_COMPLETIONS, no_repeat_ngram_size=0, do_sample=True, temperature=1.0)

# decodes n prints
for i in range(N_COMPLETIONS):
    output = tokenizer.decode(output_ids[i], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(output)

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}



Kevin is my friend. His name is Kevin.
Kevin is my friend. His name is Kevin.
Kevin is my friend. His name is Kevin.
Kevin is my friend. His name is Kevin.
Kevin is my friend. His name is Kevin.
Kevin is my friend. His name is Kevin.
Kevin is my friend. His name is Kevin.
Kevin is my friend. His name is Kevin.
Kevin is my friend. His name is Kevin.
Kevin is my friend. His name is Kevin.
